# Deployments

You now know how to package your app components into containers, group them into pods, provide them with temporary or permanent storage, pass both secret and non-secret config data to them, and allow pods to find and talk to each other. You know how to run a full-fledged system composed of independently running smaller components—microservices, if you will. Is there anything else?

Eventually, you’re going to want to update your app. This chapter covers how to update apps running in a Kubernetes cluster and how Kubernetes helps you move toward a true zero-downtime update process. Although this can be achieved using only ReplicationControllers or ReplicaSets, Kubernetes also provides a Deployment resource that sits on top of ReplicaSets and enables declarative application updates. If you’re not completely sure what that means, keep reading—it’s not as complicated as it sounds.

## Updating applications running in pods

Let’s start off with a simple example. Imagine having a set of pod instances providing a service to other pods and/or external clients. After reading this book up to this point, you likely recognize that these pods are backed by a ReplicationController or a ReplicaSet. A Service also exists through which clients (apps running in other pods or external clients) access the pods. This is how a basic application looks in Kubernetes (shown in figure 9.1).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/09fig01.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_9-1.png" class="medium-zoom-image">

Initially, the pods run the first version of your application—let’s suppose its image is tagged as v1. You then develop a newer version of the app and push it to an image repository as a new image, tagged as v2. You’d next like to replace all the pods with this new version. Because you can’t change an existing pod’s image after the pod is created, you need to remove the old pods and replace them with new ones running the new image.

You have two ways of updating all those pods. You can do one of the following:
- Delete all existing pods first and then start the new ones.
- Start new ones and, once they’re up, delete the old ones. You can do this either by adding all the new pods and then deleting all the old ones at once, or sequentially, by adding new pods and removing old ones gradually.

Both these strategies have their benefits and drawbacks. The first option would lead to a short period of time when your application is unavailable. The second option requires your app to handle running two versions of the app at the same time. If your app stores data in a data store, the new version shouldn’t modify the data schema or the data in such a way that breaks the previous version.

**Pushing updates to the same image tag**

Modifying an app and pushing the changes to the same image tag isn’t a good idea, but we all tend to do that during development. If you’re modifying the latest tag, that’s not a problem, but when you’re tagging an image with a different tag (for example, tag v1 instead of latest), once the image is pulled by a worker node, the image will be stored on the node and not pulled again when a new pod using the same image is run (at least that’s the default policy for pulling images).

That means any changes you make to the image won’t be picked up if you push them to the same tag. If a new pod is scheduled to the same node, the Kubelet will run the old version of the image. On the other hand, nodes that haven’t run the old version will pull and run the new image, so you might end up with two different versions of the pod running. To make sure this doesn’t happen, you need to set the container’s imagePullPolicy property to Always.

You need to be aware that the default imagePullPolicy depends on the image tag. If a container refers to the latest tag (either explicitly or by not specifying the tag at all), imagePullPolicy defaults to Always, but if the container refers to any other tag, the policy defaults to IfNotPresent.

When using a tag other than latest, you need to set the imagePullPolicy properly if you make changes to an image without changing the tag. Or better yet, make sure you always push changes to an image under a new tag.

## Using Deployments for updating apps declaratively

A Deployment is a higher-level resource meant for deploying applications and updating them declaratively, instead of doing it through a ReplicationController or a ReplicaSet, which are both considered lower-level concepts.

When you create a Deployment, a ReplicaSet resource is created underneath (eventually more of them). As you may remember from chapter 4, ReplicaSets are a new generation of ReplicationControllers, and should be used instead of them. Replica-Sets replicate and manage pods, as well. When using a Deployment, the actual pods are created and managed by the Deployment’s ReplicaSets, not by the Deployment directly (the relationship is shown in figure 9.8).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/09fig08.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_9-8.png" class="medium-zoom-image">

You might wonder why you’d want to complicate things by introducing another object on top of a ReplicationController or ReplicaSet, when they’re what suffices to keep a set of pod instances running. As the rolling update example in section 9.2 demonstrates, when updating the app, you need to introduce an additional ReplicationController and coordinate the two controllers to dance around each other without stepping on each other’s toes. You need something coordinating this dance. A Deployment resource takes care of that (it’s not the Deployment resource itself, but the controller process running in the Kubernetes control plane that does thatm

Using a Deployment instead of the lower-level constructs makes updating an app much easier, because you’re defining the desired state through the single Deployment resource and letting Kubernetes take care of the rest, as you’ll see in the next few pages.

### Creating a Deployment

Creating a Deployment isn’t that different from creating a ReplicationController. A Deployment is also composed of a label selector, a desired replica count, and a pod template. In addition to that, it also contains a field, which specifies a deployment strategy that defines how an update should be performed when the Deployment resource is modified.

**Creating a Deployment manifest**

Let’s see how to use the kubia-v1 ReplicationController example from earlier in this chapter and modify it so it describes a Deployment instead of a ReplicationController. As you’ll see, this requires only three trivial changes. The following listing shows the modified YAML.

> Nardimo image za verzijo v1:
- `docker build -t leon11sj/kubia:v1 .`
- `docker push leon11sj/kubia:v1`

```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: kubia
spec:
  replicas: 3
  template:
    metadata:
      name: kubia
      labels:
        app: kubia
    spec:
      containers:
      - image: leon11sj/kubia:v1
        name: nodejs
  selector:
    matchLabels:
      app: kubia
```

**Creating the Deployment resource**

You’re now ready to create the Deployment:

    kubectl apply -f ex01-kubia-deployment-v1.yaml --record

```yml

apiVersion: v1
kind: Service
metadata:
  name: kubia
spec:
  type: LoadBalancer
  selector:
    app: kubia
  ports:
  - port: 80
    targetPort: 8080
```

    kubectl apply -f ex01-service.yaml

**Displaying the status of the deployment rollout**

You can use the usual `kubectl get deployment` and the `kubectl describe deployment` commands to see details of the Deployment, but let me point you to an additional command, which is made specifically for checking a Deployment’s status:

    kubectl rollout status deployment kubia

According to this, the Deployment has been successfully rolled out, so you should see the three pod replicas up and running. Let’s see:

    kubectl get po

**Understanding how Deployments create ReplicaSets, which then create the pods**

Take note of the names of these pods. Earlier, when you used a ReplicationController to create pods, their names were composed of the name of the controller plus a randomly generated string (for example, kubia-v1-m33mv). The three pods created by the Deployment include an additional numeric value in the middle of their names. What is that exactly?

The number corresponds to the hashed value of the pod template in the Deployment and the ReplicaSet managing these pods. As we said earlier, a Deployment doesn’t manage pods directly. Instead, it creates ReplicaSets and leaves the managing to them, so let’s look at the ReplicaSet created by your Deployment:

    kubectl get replicasets

The ReplicaSet’s name also contains the hash value of its pod template. As you’ll see later, a Deployment creates multiple ReplicaSets—one for each version of the pod template. Using the hash value of the pod template like this allows the Deployment to always use the same (possibly existing) ReplicaSet for a given version of the pod template.

**Accessing the pods through the service**

With the three replicas created by this ReplicaSet now running, you can use the Service you created a while ago to access them, because you made the new pods’ labels match the Service’s label selector.

Up until this point, you probably haven’t seen a good-enough reason why you should use Deployments over ReplicationControllers. Luckily, creating a Deployment also hasn’t been any harder than creating a ReplicationController. Now, you’ll start doing things with this Deployment, which will make it clear why Deployments are superior. This will become clear in the next few moments, when you see how updating the app through a Deployment resource compares to updating it through a ReplicationController.

### Updating a Deployment

Previously, when you ran your app using a ReplicationController, you had to explicitly tell Kubernetes to perform the update by running `kubectl rolling-update`. You even had to specify the name for the new ReplicationController that should replace the old one. Kubernetes replaced all the original pods with new ones and deleted the original ReplicationController at the end of the process. During the process, you basically had to stay around, keeping your terminal open and waiting for kubectl to finish the rolling update.

Now compare this to how you’re about to update a Deployment. The only thing you need to do is modify the pod template defined in the Deployment resource and Kubernetes will take all the steps necessary to get the actual system state to what’s defined in the resource. Similar to scaling a ReplicationController or ReplicaSet up or down, all you need to do is reference a new image tag in the Deployment’s pod template and leave it to Kubernetes to transform your system so it matches the new desired state.

**Understanding the available deployment strategies**

How this new state should be achieved is governed by the deployment strategy configured on the Deployment itself. The default strategy is to perform a rolling update (the strategy is called `RollingUpdate`). The alternative is the `Recreate` strategy, which deletes all the old pods at once and then creates new ones, similar to modifying a ReplicationController’s pod template and then deleting all the pods (we talked about this in section 9.1.1).
- The `Recreate` strategy causes all old pods to be deleted before the new ones are created. Use this strategy when your application doesn’t support running multiple versions in parallel and requires the old version to be stopped completely before the new one is started. This strategy does involve a short period of time when your app becomes completely unavailable.

- The `RollingUpdate` strategy, on the other hand, removes old pods one by one, while adding new ones at the same time, keeping the application available throughout the whole process, and ensuring there’s no drop in its capacity to handle requests. This is the default strategy. The upper and lower limits for the number of pods above or below the desired replica count are configurable. You should use this strategy only when your app can handle running both the old and new version at the same time.

**Slowing down the rolling update for demo purposes**

In the next exercise, you’ll use the RollingUpdate strategy, but you need to slow down the update process a little, so you can see that the update is indeed performed in a rolling fashion. You can do that by setting the minReadySeconds attribute on the Deployment. We’ll explain what this attribute does by the end of this chapter. For now, set it to 10 seconds with the kubectl patch command.

    kubectl patch deployment kubia -p '{"spec": {"minReadySeconds": 10}}'

> The kubectl patch command is useful for modifying a single property or a limited number of properties of a resource without having to edit its definition in a text editor.

You used the patch command to change the spec of the Deployment. This doesn’t cause any kind of update to the pods, because you didn’t change the pod template. Changing other Deployment properties, like the desired replica count or the deployment strategy, also doesn’t trigger a rollout, because it doesn’t affect the existing individual pods in any way.

**Triggering the rolling update**

If you’d like to track the update process as it progresses, first run the curl loop again in another terminal to see what’s happening with the requests (don’t forget to replace the IP with the actual external IP of your service):

    while true; do curl http://35.242.218.251/; sleep 0.3; done

> Nardimo image za verzijo v2:
- `docker build -t leon11sj/kubia:v2 .`
- `docker push leon11sj/kubia:v2`

To trigger the actual rollout, you’ll change the image used in the single pod container to luksa/kubia:v2. Instead of editing the whole YAML of the Deployment object or using the patch command to change the image, you’ll use the kubectl `set image command`, which allows changing the image of any resource that contains a container (ReplicationControllers, ReplicaSets, Deployments, and so on). You’ll use it to modify your Deployment like this:

    kubectl set image deployment kubia nodejs=leon11sj/kubia:v2

When you execute this command, you’re updating the kubia Deployment’s pod template so the image used in its nodejs container is changed to luksa/kubia:v2 (from :v1). This is shown in figure 9.9.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/09fig09_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_9-9.png" class="medium-zoom-image">

**Ways of modifying Deployments and other resources**

Over the course of this book, you’ve learned several ways how to modify an existing object. Let’s list all of them together to refresh your memory.

<img style="-webkit-user-select: none;margin: auto;cursor: zoom-in;" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/table_9-1.png" width="1159" height="938">

All these methods are equivalent as far as Deployments go. What they do is change the Deployment’s specification. This change then triggers the rollout process.

If you’ve run the curl loop, you’ll see requests initially hitting only the v1 pods; then more and more of them hit the v2 pods until, finally, all of them hit only the remaining v2 pods, after all v1 pods are deleted. This works much like the rolling update performed by kubectl.



**Understanding the awesomeness of Deployments**

Let’s think about what has happened. By changing the pod template in your Deployment resource, you’ve updated your app to a newer version—by changing a single field!

The controllers running as part of the Kubernetes control plane then performed the update. The process wasn’t performed by the kubectl client, like it was when you used kubectl rolling-update. I don’t know about you, but I think that’s simpler than having to run a special command telling Kubernetes what to do and then waiting around for the process to be completed.

> NOTE: Be aware that if the pod template in the Deployment references a ConfigMap (or a Secret), modifying the ConfigMap will not trigger an update. One way to trigger an update when you need to modify an app’s config is to create a new ConfigMap and modify the pod template so it references the new ConfigMap.

The events that occurred below the Deployment’s surface during the update are similar to what happened during the kubectl rolling-update. An additional ReplicaSet was created and it was then scaled up slowly, while the previous ReplicaSet was scaled down to zero (the initial and final states are shown in figure 9.10).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/09fig10_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_9-10.png" class="medium-zoom-image">

You can still see the old ReplicaSet next to the new one if you list them:

    kubectl get rs

Similar to ReplicationControllers, all your new pods are now managed by the new ReplicaSet. Unlike before, the old ReplicaSet is still there, whereas the old Replication-Controller was deleted at the end of the rolling-update process. You’ll soon see what the purpose of this inactive ReplicaSet is.

But you shouldn’t care about ReplicaSets here, because you didn’t create them directly. You created and operated only on the Deployment resource; the underlying ReplicaSets are an implementation detail. You’ll agree that managing a single Deployment object is much easier compared to dealing with and keeping track of multiple ReplicationControllers.

Although this difference may not be so apparent when everything goes well with a rollout, it becomes much more obvious when you hit a problem during the rollout process. Let’s simulate one problem right now.

### Rolling back a deployment

You’re currently running version v2 of your image, so you’ll need to prepare version 3 first.

**Creating version 3 of your app**

In version 3, you’ll introduce a bug that makes your app handle only the first four requests properly. All requests from the fifth request onward will return an internal server error (HTTP status code 500). You’ll simulate this by adding an if statement at the beginning of the handler function. The following listing shows the new code, with all required changes shown in bold.

    const http = require('http');
    const os = require('os');

    var requestCount = 0;

    console.log("Kubia server starting...");

    var handler = function(request, response) {
      console.log("Received request from " + request.connection.remoteAddress);
      if (++requestCount >= 5) {
        response.writeHead(500);
        response.end("Some internal error has occurred! This is pod " + os.hostname() + "\n");
        return;
      }
      response.writeHead(200);
      response.end("This is v3 running in pod " + os.hostname() + "\n");
    };

    var www = http.createServer(handler);
    www.listen(8080);



As you can see, on the fifth and all subsequent requests, the code returns a 500 error with the message “Some internal error has occurred...”

> Nardimo image za verzijo v3:
- `docker build -t leon11sj/kubia:v3 .`
- `docker push leon11sj/kubia:v3`

**Deploying version 3**

I’ve made the v3 version of the image available as luksa/kubia:v3. You’ll deploy this new version by changing the image in the Deployment specification again:



    kubectl apply -f ex03-kubia-deployment-v3.yaml

You can follow the progress of the rollout with kubectl rollout status:

    kubectl rollout status deployment kubia

The new version is now live. As the following listing shows, after a few requests, your web clients start receiving errors.

**Undoing a rollout**

You can’t have your users experiencing internal server errors, so you need to do something about it fast. In section 9.3.6 you’ll see how to block bad rollouts automatically, but for now, let’s see what you can do about your bad rollout manually. Luckily, Deployments make it easy to roll back to the previously deployed version by telling Kubernetes to undo the last rollout of a Deployment:

    kubectl rollout undo deployment kubia

This rolls the Deployment back to the previous revision.

> TIP: The undo command can also be used while the rollout process is still in progress to essentially abort the rollout. Pods already created during the rollout process are removed and replaced with the old ones again.

**Displaying a Deployment’s rollout history**

Rolling back a rollout is possible because Deployments keep a revision history. As you’ll see later, the history is stored in the underlying ReplicaSets. When a rollout completes, the old ReplicaSet isn’t deleted, and this enables rolling back to any revision, not only the previous one. The revision history can be displayed with the kubectl rollout history command:

    kubectl rollout history deployment kubia

Remember the `--record` command-line option you used when creating the Deployment? Without it, the CHANGE-CAUSE column in the revision history would be empty, making it much harder to figure out what’s behind each revision.

**Rolling back to a specific Deployment revision**

You can roll back to a specific revision by specifying the revision in the undo command. For example, if you want to roll back to the first version, you’d execute the following command:

    kubectl rollout undo deployment kubia --to-revision=1

Remember the inactive ReplicaSet left over when you modified the Deployment the first time? The ReplicaSet represents the first revision of your Deployment. All Replica-Sets created by a Deployment represent the complete revision history, as shown in figure 9.11. Each ReplicaSet stores the complete information of the Deployment at that specific revision, so you shouldn’t delete it manually. If you do, you’ll lose that specific revision from the Deployment’s history, preventing you from rolling back to it.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/09fig11_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_9-11.png" class="medium-zoom-image">

But having old ReplicaSets cluttering your ReplicaSet list is not ideal, so the length of the revision history is limited by the revisionHistoryLimit property on the Deployment resource. It defaults to two, so normally only the current and the previous revision are shown in the history (and only the current and the previous ReplicaSet are preserved). Older ReplicaSets are deleted automatically.



### Blocking rollouts of bad versions

Before you conclude this chapter, we need to discuss one more property of the Deployment resource. Remember the minReadySeconds property you set on the Deployment at the beginning of section 9.3.2? You used it to slow down the rollout, so you could see it was indeed performing a rolling update and not replacing all the pods at once. The main function of minReadySeconds is to prevent deploying malfunctioning versions, not slowing down a deployment for fun.

**Understanding the applicability of minReadySeconds**

The minReadySeconds property specifies how long a newly created pod should be ready before the pod is treated as available. Until the pod is available, the rollout process will not continue (remember the maxUnavailable property?). A pod is ready when readiness probes of all its containers return a success. If a new pod isn’t functioning properly and its readiness probe starts failing before minReadySeconds have passed, the rollout of the new version will effectively be blocked.

You used this property to slow down your rollout process by having Kubernetes wait 10 seconds after a pod was ready before continuing with the rollout. Usually, you’d set minReadySeconds to something much higher to make sure pods keep reporting they’re ready after they’ve already started receiving actual traffic.

Although you should obviously test your pods both in a test and in a staging environment before deploying them into production, using minReadySeconds is like an airbag that saves your app from making a big mess after you’ve already let a buggy version slip into production.

With a properly configured readiness probe and a proper minReadySeconds setting, Kubernetes would have prevented us from deploying the buggy v3 version earlier. Let me show you how.

**Defining a readiness probe to prevent our v3 version from being rolled out fully**

You’re going to deploy version v3 again, but this time, you’ll have the proper readiness probe defined on the pod. Your Deployment is currently at version v4, so before you start, roll back to version v2 again so you can pretend this is the first time you’re upgrading to v3. If you wish, you can go straight from v4 to v3, but the text that follows assumes you returned to v2 first.

    kubectl apply -f ex02-kubia-deployment-v2.yaml

Unlike before, where you only updated the image in the pod template, you’re now also going to introduce a readiness probe for the container at the same time. Up until now, because there was no explicit readiness probe defined, the container and the pod were always considered ready, even if the app wasn’t truly ready or was returning errors. There was no way for Kubernetes to know that the app was malfunctioning and shouldn’t be exposed to clients.

To change the image and introduce the readiness probe at once, you’ll use the kubectl apply command. You’ll use the following YAML to update the deployment (you’ll store it as kubia-deployment-v3-with-readinesscheck.yaml), as shown in the following listing.

**Updating a Deployment with kubectl apply**

To update the Deployment this time, you’ll use kubectl apply like this:

    kubectl apply -f ex03-kubia-deployment-v3-with-readinesscheck.yaml

The apply command updates the Deployment with everything that’s defined in the YAML file. It not only updates the image but also adds the readiness probe definition and anything else you’ve added or modified in the YAML. If the new YAML also contains the replicas field, which doesn’t match the number of replicas on the existing Deployment, the apply operation will also scale the Deployment, which isn’t usually what you want.

> TIP: To keep the desired replica count unchanged when updating a Deployment with kubectl apply, don’t include the replicas field in the YAML.

Running the apply command will kick off the update process, which you can again follow with the rollout status command:

    kubectl rollout status deployment kubia

Because the status says one new pod has been created, your service should be hitting it occasionally, right? Let’s see:

Nope, you never hit the v3 pod. Why not? Is it even there? List the pods:

    kubectl get po

Aha! There’s your problem (or as you’ll learn soon, your blessing)! The pod is shown as not ready, but I guess you’ve been expecting that, right? What has happened?

**Understanding how a readiness probe prevents bad versions from being rolled out**

As soon as your new pod starts, the readiness probe starts being hit every second (you set the probe’s interval to one second in the pod spec). On the fifth request the readiness probe began failing, because your app starts returning HTTP status code 500 from the fifth request onward.

As a result, the pod is removed as an endpoint from the service (see figure 9.14). By the time you start hitting the service in the curl loop, the pod has already been marked as not ready. This explains why you never hit the new pod with curl. And that’s exactly what you want, because you don’t want clients to hit a pod that’s not functioning properly.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/09fig14_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_9-14.png" class="medium-zoom-image">

But what about the rollout process? The rollout status command shows only one new replica has started. Thankfully, the rollout process will not continue, because the new pod will never become available. To be considered available, it needs to be ready for at least 10 seconds. Until it’s available, the rollout process will not create any new pods, and it also won’t remove any original pods because you’ve set the maxUnavailable property to 0.

The fact that the deployment is stuck is a good thing, because if it had continued replacing the old pods with the new ones, you’d end up with a completely non-working service, like you did when you first rolled out version 3, when you weren’t using the readiness probe. But now, with the readiness probe in place, there was virtually no negative impact on your users. A few users may have experienced the internal server error, but that’s not as big of a problem as if the rollout had replaced all pods with the faulty version 3.

> TIP: If you only define the readiness probe without setting minReadySeconds properly, new pods are considered available immediately when the first invocation of the readiness probe succeeds. If the readiness probe starts failing shortly after, the bad version is rolled out across all pods. Therefore, you should set minReadySeconds appropriately.

**Aborting a bad rollout**

Because the rollout will never continue, the only thing to do now is abort the rollout by undoing it:

    kubectl rollout undo deployment kubia

> NOTE: In future versions, the rollout will be aborted automatically when the time specified in progressDeadlineSeconds is exceeded.

    kubectl delete service kubia
    kubectl delete deployments kubia